In [1]:
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import io
from typing import Optional, Union, List


from datafin.aws import SecretsClient                      #type: ignore
from datafin.aws import S3Client                           #type: ignore

In [2]:
class GmailClient:
    """
    A Gmail email client that uses SMTP to send emails with optional DataFrame attachments.
    
    Usage:
        client = GmailClient("your_email@gmail.com", "your_app_password")
        client.send_email("recipient@example.com", "Subject", "Email body", df=dataframe)
    """
    
    def __init__(self, sender_email: str, app_password: str):
        """
        Initialize the Gmail client with credentials.
        
        Args:
            sender_email (str): Your Gmail address
            app_password (str): Your Gmail app password (not regular password)
        
        Note:
            You need to enable 2FA on your Gmail account and generate an app password.
            Regular Gmail passwords won't work due to security restrictions.
        """
        self.sender_email = sender_email
        self.app_password = app_password
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        
        # Test connection on initialization
        self._test_connection()
    
    def _test_connection(self) -> bool:
        """
        Test the SMTP connection to ensure credentials are valid.
        
        Returns:
            bool: True if connection successful
            
        Raises:
            Exception: If connection fails
        """
        try:
            server = smtplib.SMTP(self.smtp_server, self.smtp_port)
            server.starttls()
            server.login(self.sender_email, self.app_password)
            server.quit()
            print("✓ Gmail SMTP connection successful")
            return True
        except Exception as e:
            raise Exception(f"Failed to connect to Gmail SMTP: {str(e)}")
    
    def send_email(self, 
                   to: Union[str, List[str]], 
                   subject: str, 
                   text: str, 
                   df: Optional[pd.DataFrame] = None,
                   csv_filename: Optional[str] = None,
                   cc: Optional[Union[str, List[str]]] = None,
                   bcc: Optional[Union[str, List[str]]] = None) -> bool:
        """
        Send an email with optional DataFrame attachment as CSV.
        
        Args:
            to (str or list): Recipient email address(es)
            subject (str): Email subject
            text (str): Email body text
            df (pd.DataFrame, optional): DataFrame to attach as CSV
            csv_filename (str, optional): Name for CSV attachment (default: 'data.csv')
            cc (str or list, optional): CC recipients
            bcc (str or list, optional): BCC recipients
            
        Returns:
            bool: True if email sent successfully
            
        Raises:
            Exception: If email sending fails
        """
        try:
            # Create message
            msg = MIMEMultipart()
            msg['From'] = self.sender_email
            msg['To'] = ', '.join(to) if isinstance(to, list) else to
            msg['Subject'] = subject
            
            # Add CC and BCC if provided
            if cc:
                msg['Cc'] = ', '.join(cc) if isinstance(cc, list) else cc
            
            # Add body text
            msg.attach(MIMEText(text, 'plain'))
            
            # Add DataFrame as CSV attachment if provided
            if df is not None:
                self._attach_dataframe_as_csv(msg, df, csv_filename)
            
            # Prepare recipient list
            recipients = []
            if isinstance(to, list):
                recipients.extend(to)
            else:
                recipients.append(to)
            
            if cc:
                if isinstance(cc, list):
                    recipients.extend(cc)
                else:
                    recipients.append(cc)
            
            if bcc:
                if isinstance(bcc, list):
                    recipients.extend(bcc)
                else:
                    recipients.append(bcc)
            
            # Send email
            server = smtplib.SMTP(self.smtp_server, self.smtp_port)
            server.starttls()
            server.login(self.sender_email, self.app_password)
            
            text_msg = msg.as_string()
            server.sendmail(self.sender_email, recipients, text_msg)
            server.quit()
            
            print(f"✓ Email sent successfully to recipient")
            return True
            
        except Exception as e:
            raise Exception(f"Failed to send email: {str(e)}")
    
    def _attach_dataframe_as_csv(self, msg: MIMEMultipart, df: pd.DataFrame, filename: Optional[str] = None) -> None:
        """
        Attach a pandas DataFrame as a CSV file to the email.
        
        Args:
            msg (MIMEMultipart): Email message object
            df (pd.DataFrame): DataFrame to attach
            filename (str, optional): CSV filename
        """
        if filename is None:
            filename = "data.csv"
        
        if not filename.endswith('.csv'):
            filename += '.csv'
        
        # Convert DataFrame to CSV string
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        csv_data = csv_buffer.getvalue()
        
        # Create attachment
        attachment = MIMEBase('application', 'octet-stream')
        attachment.set_payload(csv_data.encode())
        encoders.encode_base64(attachment)
        
        # Add header
        attachment.add_header(
            'Content-Disposition',
            f'attachment; filename= {filename}'
        )
        
        msg.attach(attachment)
        print(f"✓ DataFrame attached as {filename} ({len(df)} rows, {len(df.columns)} columns)")

In [3]:
secrets = SecretsClient()

s3 = S3Client(
    bucket_name = secrets.get_bucket_name(),
    aws_access_key_id = secrets.aws_access_key,
    aws_secret_access_key = secrets.aws_secret_access_key,
)

df = s3.get_parquet(
    path = 'dev/polygon/equities/spy/store/market-open/year=2025',
    file_name = 'spy-open-2025-06-11'
)

client = GmailClient(secrets.get_gmail_address(), secrets.get_gmail_app_password())


client.send_email(
    to=secrets.get_gmail_send_to_address(),
    subject="Monthly Report with Data",
    text="Please find the monthly data report attached as a CSV file.\n\nBest regards,\nYour Name",
    df=df,
    csv_filename="monthly_report.csv"
)

✓ Gmail SMTP connection successful
✓ DataFrame attached as monthly_report.csv (60 rows, 9 columns)
✓ Email sent successfully to recipient


True

In [4]:
s = SecretsClient()

s3 = S3Client(
    bucket_name = s.get_bucket_name(),
    aws_access_key_id = s.aws_access_key,
    aws_secret_access_key = s.aws_secret_access_key
)

df = s3.get_parquet(
    path = 'dev/polygon/equities/spy/store/market-open/year=2025',
    file_name = 'spy-open-2025-06-12'
)

In [10]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

fig = go.Figure(data=[
        go.Candlestick(
            x=df['datetime_ny'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close']
        )
    ]
)


    

fig.update_layout(xaxis_rangeslider_visible=False)
fig.write_image(
    "candlestick_chart.png",
    scale = 4, 
    engine="kaleido"     # Best engine for high quality
)